In [1]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

## CSV Parser

In [6]:
model = ChatOpenAI(temperature=0)

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template='List five places {places}.\n{format_instructions}',
    input_variables=["places"],
    partial_variables={'format_instructions': format_instructions}
)

chain = prompt | model | output_parser

In [7]:
chain.invoke({'places': 'for summer tourism in Brazil'})

['Rio de Janeiro',
 'Salvador',
 'Florianópolis',
 'Foz do Iguaçu',
 'Amazon Rainforest']

# JSON Parser

In [8]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from openai import BaseModel
from pydantic import Field

class Travel(BaseModel):
    place: str = Field(description='anme of the places')
    description: str = Field(description='description of the place')
    activities: str = Field(description='what to do in that place')

travel_query = "Suggest a place in Brazil for going on a trip this summer to avoid heat."

parser = JsonOutputParser(pydantic_object=Travel)

prompt = PromptTemplate(
    template='Answer the user query. \n{format_instructions}\n{query}\n',
    input_variables=['query'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

chain = prompt | model | parser

chain.invoke({"query": travel_query})

{'place': 'Gramado',
 'description': 'A charming mountain town known for its cooler temperatures and European-inspired architecture.',
 'activities': 'Visiting the Mini Mundo miniature park, exploring the Black Lake Park, and enjoying the local cuisine and chocolate shops.'}

## Without pydantic

In [9]:
travel_query = "Suggest a place in Brazil for going on a trip this summer to avoid heat."

travel_query = "Suggest a place in Brazil for going on a trip this summer to avoid heat."

parser = JsonOutputParser()

prompt = PromptTemplate(
    template='Answer the user query. \n{format_instructions}\n{query}\n',
    input_variables=['query'],
    partial_variables={'format_instructions': parser.get_format_instructions()}
)

chain = prompt | model | parser

chain.invoke({"query": travel_query})

{'destination': 'Campos do Jordão',
 'description': 'Campos do Jordão is a mountain town in the state of São Paulo, Brazil. It is known for its cool climate and European-style architecture, making it a popular destination for those looking to escape the heat during the summer months. The town is surrounded by lush forests and offers a variety of outdoor activities such as hiking, horseback riding, and cycling. Campos do Jordão also has a vibrant cultural scene, with numerous art galleries, museums, and music festivals to explore. Overall, it is a great place to visit for a refreshing summer getaway.'}

## Structured Output parser

In [10]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

responses_schemas = [
    ResponseSchema(name="answer", description='answer to the user\'s questions'),
    ResponseSchema(name='description', description='detailed description on the answer topic'),
    ResponseSchema(name='applications', description='Real world applications of the aswer topic')
]

output_parser = StructuredOutputParser.from_response_schemas(responses_schemas)

format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template='answer the users question as best as possible.\n{format_instructions}\n{question}',
    input_variables=['questions'],
    partial_variables={'format_instructions': format_instructions}
)

chain = prompt | model | output_parser

chain.invoke({'question': 'Name an invetion in Healthcare that has caused revolution in twenty first century'})

{'answer': 'Telemedicine',
 'description': 'Telemedicine is the use of telecommunication and information technology to provide clinical healthcare from a distance. It has revolutionized healthcare by enabling remote diagnosis, monitoring, and treatment of patients, especially in rural or underserved areas. Telemedicine has improved access to healthcare services, reduced healthcare costs, and increased efficiency in healthcare delivery.',
 'applications': 'Real world applications of telemedicine include virtual doctor visits, remote patient monitoring, telepsychiatry, telestroke services, and tele-ICU care.'}